Данный скрипт предназначен для вычисления ряда акустических параметров пения птиц. На вход подаётся директория, содержащая логи с данными, полученные из приложения Raven Ligth (Selection Tables). На выходе получаются 4 файла: файл с рассчитанными параметрами (parameters) и файл с описательными статистиками данных параметров, оба в двух форматах (csv и xls). Расчитываемые параметры каждого лога: длительность, общее кол-во песен, число типов песен (ЕР) в записи (diversity), медианную длительность песен, медианную длительность пауз между песнями, частоту смены напева, Slin, Scons, RE1, versatility. Язык скрипта - Python 3.10, используемые библиотеки: Pandas, Numpy, Os, Statistics.

In [1]:
import pandas as pd
import numpy as np
import statistics
import os

In [2]:
# Здесь необходимо прописать рабочую директорию в формате 'C:\\Users\\..' 

home = 'C:\\Users\\..' 
os.chdir(home)

In [3]:
data_files = []   
for i in os.listdir(home):
    if 'txt' in i:
        data_files.append(i)

# Ищем файлы с логами в директории. Т.к. скрипт написан под Selection Tables от Raven Ligth, то и поиск идёт по расширению txt.

In [4]:
# Преобразуем найденные файлы в датафреймы и соберём в единый список.

data_files_df = []

for file in data_files:
    df = pd.read_csv(file, sep = '\t')
    name = file.split('.')[0]
    df['name'] = name
    data_files_df.append(df)

In [5]:
def newname(x):
    x = x.lower()
    return x
    # функция для перевода наименования в нижний регистр    

In [6]:
parameters = pd.DataFrame() # создадим пустой датафрейм, куда будут добавляться параметры по каждому логу

for df in data_files_df:
    df = df.rename(columns = {'Begin Time (s)':'begin_time', 'End Time (s)':'end_time', 'Low Freq (Hz)':'low_freq', 
                              'High Freq (Hz)':'high_freq', 'Delta Time (s)':'delta_time',  'Delta Freq (Hz)':'delta_freq', 
                              'Avg Power Density (dB FS/Hz)':'power'}).rename(columns = newname)

    total_songs = df.shape[0]


    trans = df.annotation.to_list() # переводим столбец с ЕР в список, дальше итерируемся по нему 
    total = 0
    total_trans = (total_songs-1) # df.shape[0] - кол-во строк, т.е. наблюдений. Всего переходов = кол-во наблюдений-1
    for i in range(len(trans)-1):
        if trans[i] != trans[i+1]:
            total += 1             # переменная total отображает кол-во переходов к ЕР другого типа
    switching_rate = total / total_trans 

    diversity = df.annotation.nunique() # размер репертуара в записи (кол-во разных типов ЕР)

    df1 = df[['selection', 'annotation']] # сохраним новую таблицу для расчётов в отдельной переменной

    df1['annotation2'] = df.annotation.shift(-1) # для подсчёта переходов создадим второй столбец со сдвигом

    df1 = df1.dropna()
    # уберём последнее значение как пропущенное, сохраним новую таблицу в отдельной переменной

    df1.annotation2 = df1.annotation2.astype(int) # поправим тип переменной - уберём нули

    transactions = df1.groupby(['annotation', 'annotation2'], as_index = False).agg({'selection' : 'count'})\
    .pivot(index='annotation', columns='annotation2', values='selection' ).fillna(0) # получаем матрицу переходов

    n = transactions.shape[0] # размерность матрицы
    # если у нас матрица не квадратная (т.е. для первого ЕР нет на него перехода), 
    # то необходимо искусственно добавить лишний столбец к матрице, иначе будет ошибка.
    if transactions.shape[0] != transactions.shape[1]:  
        transactions.insert(0, "0", 0)

    matrix = transactions.to_numpy() # сохраним датафрейм с переходами как матрицу

    count = 0
    for i in range(n):
        for j in range(n):
            if matrix[i][j] != 0:
                count += 1 # вычисляем кол-во типов переходов
    if count != 0:
        SLIN = diversity / count
    else:
        SLIN = 1

    sum_freq_trans = 0

    for l in matrix.tolist():
        sum_freq_trans += max(l)  # ∑ наиболее частых переходов (для каждого типа ЕР)

    SCONS = sum_freq_trans / total_trans

    E0  = -diversity**2*(1/diversity * np.log2(1/diversity))  # формула Шеннона для E0 

    # Фукция для расчёта переходных вероятностей

    def transition_matrix(transitions):
        n = 1+ max(transitions) #number of states

        M = [[0]*n for _ in range(n)]

        for (i,j) in zip(transitions,transitions[1:]):
            M[i][j] += 1

        #now convert to probabilities:
        for row in M:
            s = sum(row)
            if s > 0:
                row[:] = [f/s for f in row]
        return M

    m = transition_matrix(trans)  # Матрица переходных вероятностей

    E1 = 0
    for i in range(n):
        for j in range(n):
            if m[i][j] != 0:
                E1 += m[i][j]*np.log2(m[i][j]) # вычисляем Е1 по формуле Шеннона для каждой вероятности, не равной 0. 
    E1 = -E1  
    if diversity > 1:  # В случае, если у нас всего 1 тип песни, при расчёте RE1 по стандартной формуле мы получим NaN.
        RE1=E1/E0  
    else:
        RE1=-E0   # Поэтому добавляем условие, и если репертуар состоит всего из одной песни, RE1 считаем равным Е1.

    if len(trans) >= 10:
        div = []
        for i in range(0, (len(trans) - len(trans)%10), 10): # создаём цикл от начала до последней десятки последовательности с шагом 10
            div.append(len(set(trans[i:i+10])))
        Vers = statistics.median(div)
    else:
        Vers = len(set(trans))

    fon_length =  df.end_time.iloc[-1] - df.begin_time.iloc[0] # общая длительность фонограммы

    song_rate = df.shape[0] / (fon_length / 60)  # частота пения

    df['pause_length'] = df.begin_time.shift(-1) - df.end_time # добавляем колонку с длительностью пауз

    time_param = df[['delta_time', 'pause_length']].describe() # рассчитваем временнЫе параметры

    median_lenth = time_param.delta_time.loc['50%']
    median_pause = time_param.delta_time.loc['50%']

    param = pd.DataFrame({'name': df.name.iloc[0], 'total_lenght': fon_length.round(2), 'total_songs': total_songs, 
                          'diversity':diversity, 'median_song_lenth':median_lenth.round(2),
                          'median_pause_length':median_pause.round(2), 'song_rate':song_rate.round(2), 
                          'switching_rate':round(switching_rate, 2),
                          'SLIN':round(SLIN, 2), 'SCONS':round(SCONS, 2), 'RE1':RE1.round(2), 'versatility':Vers}, index=[0])

    parameters = parameters.append(param, ignore_index=True)

In [7]:
parameters # ну посмотрим, что получилось :)

name  total_lenght  total_songs  diversity  median_song_lenth  \
0  MZ000027         11.92            4          1               0.51   
1  MZ000037        104.33           21         10               0.90   
2  MZ000073        657.72          123          8               0.80   
3  MZ000077         63.64           11          1               1.06   
4  MZ000092        336.78           37          5               0.85   
5  MZ000123        489.84           25          3               0.77   

   median_pause_length  song_rate  switching_rate  SLIN  SCONS   RE1  \
0                 0.51      20.14            0.00  1.00   1.00  0.00   
1                 0.90      12.08            0.85  0.59   0.60  0.14   
2                 0.80      11.22            0.84  0.28   0.45  0.39   
3                 1.06      10.37            0.00  1.00   1.00  0.00   
4                 0.85       6.59            0.56  0.33   0.53  0.50   
5                 0.77       3.06            0.12  0.60   0.92  0.03   

   versatility  
0          1.0  
1          6.5  
2          4.0  
3          1.0  
4          3.0  
5          2.0

In [8]:
parameters.to_csv('parameters.csv') # сохраняем результаты в папку в двух форматах.
parameters.to_excel('parameters.xlsx')

In [9]:
results = parameters.describe().round(2) # рассчитываем описательные статистики и также сохраняем в файлы
results.to_csv('results.csv')
results.to_excel('results.xlsx')
results

total_lenght  total_songs  diversity  median_song_lenth  \
count          6.00         6.00       6.00               6.00   
mean         277.37        36.83       4.67               0.82   
std          260.55        43.73       3.72               0.18   
min           11.92         4.00       1.00               0.51   
25%           73.81        13.50       1.50               0.78   
50%          220.55        23.00       4.00               0.82   
75%          451.58        34.00       7.25               0.89   
max          657.72       123.00      10.00               1.06   

       median_pause_length  song_rate  switching_rate  SLIN  SCONS   RE1  \
count                 6.00       6.00            6.00  6.00   6.00  6.00   
mean                  0.82      10.58            0.40  0.63   0.75  0.18   
std                   0.18       5.77            0.40  0.31   0.25  0.22   
min                   0.51       3.06            0.00  0.28   0.45  0.00   
25%                   0.78       7.54            0.03  0.40   0.55  0.01   
50%                   0.82      10.80            0.34  0.60   0.76  0.08   
75%                   0.89      11.86            0.77  0.90   0.98  0.33   
max                   1.06      20.14            0.85  1.00   1.00  0.50   

       versatility  
count         6.00  
mean          2.92  
std           2.11  
min           1.00  
25%           1.25  
50%           2.50  
75%           3.75  
max           6.50